This code is part of a preprocessing step for static layers used in firenet. It involves loading various landfire layers, reprojecting them to a common coordinate reference system (CRS), and then stacking them together into a single multi-layered dataset. Landfire data gives spatial information such as elevation, vegetation height etc. that are useful to feed into firenet when it makes inference via interpreting spectral satellite data.

The reprojected layers are matched to the CRS of a median image to ensure spatial alignment across all layers. This combined dataset is then saved as a NetCDF file for further use. 

Memory management is also considered by manually invoking garbage collection after processing each layer to ensure the environment won't crash.


In [1]:
import gc
import xarray as xr
import rioxarray

# Load the landfire layers
landfire_tif_paths = ['/Users/adamhunter/Documents/school projs/firenet/data/static_layers/LF2020_Elev_220_CONUS/Tif/LC20_Elev_220.tif',
                        '/Users/adamhunter/Documents/school projs/firenet/data/static_layers/LF2022_CBD_220_CONUS/Tif/LC22_CBD_220.tif',
                        '/Users/adamhunter/Documents/school projs/firenet/data/static_layers/LF2022_EVC_220_CONUS/Tif/LC22_EVC_220.tif',
                        '/Users/adamhunter/Documents/school projs/firenet/data/static_layers/LF2022_EVH_220_CONUS/Tif/LC22_EVH_220.tif',
                        '/Users/adamhunter/Documents/school projs/firenet/data/static_layers/LF2022_FBFM40_220_CONUS/Tif/LC22_F40_220.tif',
                        '/Users/adamhunter/Documents/school projs/firenet/data/static_layers/LF2022_FVH_220_CONUS/Tif/LC22_FVH_220.tif']

# Initialize an empty dictionary to store the datasets
datasets = {}

In [2]:
import rioxarray
from rasterio.enums import Resampling
import os

# Define the target resolution
target_resolution = (3270.9038690767325, -3270.9038690767325)

# Create an empty dictionary to hold the reprojected/resampled datasets
landfire_datasets = {}

median_ds = rioxarray.open_rasterio('reprojected_median_ds.nc')

In [3]:
median_ds.rio.crs

CRS.from_epsg(5070)

In [4]:
# Loop through each Landfire TIFF file path
for tif_path in landfire_tif_paths[2:]:
    print(tif_path)
    # Open the TIFF file as an xarray dataset
    landfire_ds = rioxarray.open_rasterio(tif_path)
    
    # Reproject and resample the dataset to the target resolution
    # Assuming the median_ds has the target CRS and resolution
    landfire_reprojected = landfire_ds.rio.reproject_match(median_ds)
    
    del landfire_ds

    # Get the variable name from the TIFF file name
    var_name = os.path.splitext(os.path.basename(tif_path))[0]
    
    # Define the new file name
    new_file_name = f"{var_name}_reprojected.tif"
    
    # Save the reprojected/resampled dataset as a new TIFF file
    landfire_reprojected.rio.to_raster(new_file_name)

    

/Users/adamhunter/Documents/school projs/firenet/data/static_layers/LF2022_EVC_220_CONUS/Tif/LC22_EVC_220.tif
